In [16]:
import pandas as pd
import ast

In [17]:
col_list = ["Account", "Online?", "Ratings", "Zone", "restaurant type", "Reviews", "Type", "City"]

In [18]:
df = pd.read_csv('sample_dataset.csv', usecols=col_list, encoding = "ISO-8859-1")
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('?', '')
df.columns = df.columns.str.lower()

<ipython-input-18-1403644490d1>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('?', '')


In [19]:
df = df.dropna()

In [20]:
# Extract the rating
def ratings_to_float(x):
    try:
        return float(x[0:3])
    except :
        return 'not float'

In [21]:
df['overall_ratings'] = df['ratings'].apply(ratings_to_float)

In [22]:
# Cleanning
df = df[df['overall_ratings']!='not float']
df = df[(df['online']=='Yes')|(df['online']=='No')]
df = df[~df['account'].str.contains('Ã')].reset_index()

In [23]:
# unify the meals types 
types = []
for i in df.index:
    z = df['restaurant_type'][i].split(",")
    if len(z) == 2 :
        z = [elem.strip().lstrip() for elem in z]
        q = [z[1],z[0]]
        if q not in types and z not in types:
            types.append(z)
            df['restaurant_type'][i] = z[0]+", "+z[1]
        elif q not in types and z in types:
            df['restaurant_type'][i] = z[0]+", "+z[1]
        elif q in types and z not in types:
            df['restaurant_type'][i] = q[0]+", "+q[1]
    else :
        df['restaurant_type'][i] = df['restaurant_type'][i]

<ipython-input-23-cda6b4a48838>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['restaurant_type'][i] = df['restaurant_type'][i]
<ipython-input-23-cda6b4a48838>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['restaurant_type'][i] = z[0]+", "+z[1]
<ipython-input-23-cda6b4a48838>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['restaurant_type'][i] = q[0]+", "+q[1]
<ipython-input-23-cda6b4a48838>:12: SettingWithCopyWarnin

In [24]:
df['reviews'] = df['reviews'].apply(lambda x:x.replace('\\n', ""))
df["reviews"] = df['reviews'].apply(lambda x : list(ast.literal_eval(x)))

In [25]:
df['rating_bucket'] = df['overall_ratings'].apply(lambda x : "low" if x < 3 else "high")

In [26]:
def rating(x):
    try :
        return x[0][6:]
    except :
        return None
    
def comment(x):
    try :
        return x[1][6:]
    except :
        return None

In [27]:
# Unnest reviews and ratings
df = df.explode('reviews').reset_index()

df["cus_rating"] = df['reviews'].apply(rating)
df["comment"] = df['reviews'].apply(comment)

In [28]:
df = df[['account', 'online', 'zone',
       'restaurant_type', 'type', 'city', 'overall_ratings',
       'cus_rating', 'comment','rating_bucket']]
df

,account,online,zone,restaurant_type,type,city,overall_ratings,cus_rating,comment,rating_bucket
0,Jalsa,Yes,Banashankari,Casual Dining,Buffet,Banashankari,4.1,4.0,A beautiful place to dine in.The interiors ta...,high
1,Jalsa,Yes,Banashankari,Casual Dining,Buffet,Banashankari,4.1,4.0,I was here for dinner with my family on a wee...,high
2,Jalsa,Yes,Banashankari,Casual Dining,Buffet,Banashankari,4.1,2.0,Its a restaurant near to Banashankari BDA. Me...,high
3,Jalsa,Yes,Banashankari,Casual Dining,Buffet,Banashankari,4.1,4.0,We went here on a weekend and one of us had t...,high
4,Jalsa,Yes,Banashankari,Casual Dining,Buffet,Banashankari,4.1,5.0,The best thing about the place is itÃÂÃÂ...,high
...,...,...,...,...,...,...,...,...,...,...
439295,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,"ITPL Main Road, Whitefield",Bar,Pubs and bars,Whitefield,4.3,5.0,My review based on food drink serviceFood:- f...,high
439296,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,"ITPL Main Road, Whitefield",Bar,Pubs and bars,Whitefield,4.3,5.0,I visited chime bar at Sheraton hotel it was ...,high
439297,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,"ITPL Main Road, Whitefield",Bar,Pubs and bars,Whitefield,4.3,4.0,This restaurant is situated inside grand Sher...,high
439298,The Nest - The Den Bengaluru,No,"ITPL Main Road, Whitefield","Casual Dining, Bar",Pubs and bars,Whitefield,3.4,5.0,"Great ambience , looking nice good selection ...",high


In [30]:
df.to_csv('cleaned_dataset.csv')